# 0
### A)

In [942]:
from pathlib import Path
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [943]:
env_path = Path("../../code_alongs/11_data_from_API/.env")
load_dotenv(env_path)


API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()



result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [944]:
def get_departures(api):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()


def get_arrivals(api):
    url = f"https://api.resrobot.se/v2.1/arrivalBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()

In [945]:
departures = get_departures(api=API_KEY)
arrivals = get_arrivals(api=API_KEY)

In [946]:
departures.keys(), arrivals.keys()

(dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']),
 dict_keys(['Arrival', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']))

### B)

In [947]:
len(arrivals["Arrival"])

235

### C)

In [948]:
len(departures["Departure"])

239

### D)

In [949]:
departures["Departure"][0].keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [950]:
for depart in departures["Departure"]:
    if "Spårväg" in depart["name"]:
        print(f"{depart['name']}, {depart['direction']}, {depart['time']}")

Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:26:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:30:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:34:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:38:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:46:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:47:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:54:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:56:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 17:04:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 17:04:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 17:13:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 17:15:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 17:22:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 17:24:00


### E)

In [951]:
departures = departures["Departure"]

In [952]:
trams = [
    {
        "name": tram.get("name"),
        "lat": tram.get("lat"),
        "lon": tram.get("lon"),
        "direction": tram.get("direction"),
    }
    for tram in departures
    if "Spårväg" in tram.get("name")
]

In [953]:
data= pd.DataFrame(trams)

In [954]:
fig = px.scatter_mapbox(data, lat="lat", lon= "lon", hover_name="direction", mapbox_style= "open-street-map", zoom= 15)

fig.show()

### F)

# 1
### A)

# Stop Lookup

In [955]:
def get_lookup(search):
    url= f"https://api.resrobot.se/v2.1/location.name?input={search}&format=json&accessId={API_KEY}"
    return requests.get(url).json()

In [956]:
lund= get_lookup("Lund")
lund["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098038'

In [957]:
gothenburg= get_lookup("Göteborg")
gothenburg["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098001'

In [958]:
def route_planner(org, des):
    
    url= f"https://api.resrobot.se/v2.1/trip?format=json&originId={org}&destId={des}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    return requests.get(url).json()

In [959]:
trip= route_planner(740098001, 740098038)

In [960]:
trip.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [961]:
example_trip = trip["Trip"][0]

In [962]:
stops=example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Göteborg Centralstation',
  'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
  'extId': '740000002',
  'routeIdx': 0,
  'lon': 11.973479,
  'lat': 57.708895,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '16:40:00',
  'depDate': '2025-01-09',
  'depDir': 'Köpenhamn Österport (Danmark)',
  'minimumChangeDuration': 'PT15M'},
 {'name': 'Mölndal station',
  'id': 'A=1@O=Mölndal station@X=12018722@Y=57655958@U=1@L=740000315@',
  'extId': '740000315',
  'routeIdx': 1,
  'lon': 12.018722,
  'lat': 57.655958,
  'depTime': '16:48:00',
  'depDate': '2025-01-09',
  'arrTime': '16:48:00',
  'arrDate': '2025-01-09',
  'minimumChangeDuration': 'PT9M'},
 {'name': 'Kungsbacka station',
  'id': 'A=1@O=Kungsbacka station@X=12079614@Y=57490556@U=1@L=740000161@',
  'extId': '740000161',
  'routeIdx': 2,
  'lon': 12.079614,
  'lat': 57.490556,
  'depTime': '16:59:00',
  'depDate': '2025-01-09',
  'arrTime': '16:59:00',
  'arrDate': '2025-01-09',
  'minimumChangeDur

In [963]:
[stop.get("name")for stop in stops]

['Göteborg Centralstation',
 'Mölndal station',
 'Kungsbacka station',
 'Varberg station',
 'Falkenberg station',
 'Halmstad Centralstation',
 'Laholm station',
 'Båstad station',
 'Ängelholm station',
 'Helsingborg Centralstation',
 'Landskrona station',
 'Lund Centralstation']

### B)

In [964]:
sthlm= get_lookup("Stockholm")
sthlm["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098000'

In [965]:
trips = route_planner(740098001, 740098000)
trips

{'Trip': [{'Origin': {'name': 'Göteborg Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
    'extId': '740000002',
    'lon': 11.973479,
    'lat': 57.708895,
    'routeIdx': 0,
    'prognosisType': 'PROGNOSED',
    'time': '16:59:00',
    'date': '2025-01-09',
    'minimumChangeDuration': 'PT15M'},
   'Destination': {'name': 'Stockholm Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
    'extId': '740000001',
    'lon': 18.058151,
    'lat': 59.330136,
    'routeIdx': 6,
    'prognosisType': 'PROGNOSED',
    'time': '20:33:00',
    'date': '2025-01-09',
    'minimumChangeDuration': 'PT20M'},
   'ServiceDays': [{'planningPeriodBegin': '2025-01-02',
     'planningPeriodEnd': '2025-06-15',
     'sDaysR': 'inte varje dag',
     'sDaysI': '9., 10. jan',
     'sDaysB': '1F8000000000000000000000000000000000000000'}],
   'LegList': {'Leg': [{'Origin': {'name

In [966]:
trip= trips["Trip"][0]

In [967]:
stops= trip["LegList"]["Leg"][0]["Stops"]["Stop"]

In [968]:
[{stop["name"]: stop.get("arrTime")}for stop in stops]

[{'Göteborg Centralstation': None},
 {'Alingsås station': None},
 {'Herrljunga station': '17:45:00'},
 {'Falköping Centralstation': '18:01:00'},
 {'Skövde Centralstation': '18:16:00'},
 {'Flemingsberg station (Huddinge kn)': '20:21:00'},
 {'Stockholm Centralstation': '20:33:00'}]

In [969]:
coords= []
for stop in stops:
    city= get_lookup(stop.get("name"))
    city= city["stopLocationOrCoordLocation"][0]["StopLocation"]
    coords.append((city["lon"], city["lat"]))

In [970]:
coords

[(11.973479, 57.708895),
 (12.532491, 57.926857),
 (13.021279, 58.079188),
 (13.553791, 58.176101),
 (13.853195, 58.390898),
 (17.945678, 59.217959),
 (18.058151, 59.330136)]

In [971]:
data= pd.DataFrame(coords, columns=["lon", "lat"])
data

,lon,lat
0,11.973479,57.708895
1,12.532491,57.926857
2,13.021279,58.079188
3,13.553791,58.176101
4,13.853195,58.390898
5,17.945678,59.217959
6,18.058151,59.330136


In [972]:
fig= px.line_mapbox(data, "lat", "lon", mapbox_style="open-street-map", zoom= 6.5, width= 800)
fig.show()